In [1]:
# ============================================
# KoGPT2 + LoRA (긴 한국어 문장 SFT, 품질/안정 버전)
# ============================================

# 0) 외부 로깅 완전 차단 (W&B 프롬프트 방지)
!pip -q uninstall -y wandb
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

# 1) 필수 라이브러리 설치
!pip -q install -U torch transformers peft accelerate

import random, torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, set_seed
from peft import LoraConfig, get_peft_model, TaskType

# 2) 재현성 & 디바이스
SEED = 42
set_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device, "| CUDA?", torch.cuda.is_available())

# 3) 모델/토크나이저 (PAD 추가 + 임베딩 리사이즈)
base_model = "skt/kogpt2-base-v2"
tok = AutoTokenizer.from_pretrained(base_model)

pad_added = False
if tok.pad_token is None:
    tok.add_special_tokens({"pad_token": "[PAD]"})
    pad_added = True
if tok.eos_token is None:   # (거의 안 일어나지만 안전)
    tok.add_special_tokens({"eos_token": "</s>"})
    pad_added = True

model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float32)
if pad_added:
    model.resize_token_embeddings(len(tok))  # 새 토큰 반영
model.config.pad_token_id = tok.pad_token_id
model.config.use_cache = False  # 경고/메모리 이슈 완화
model.to(device)

# 4) LoRA 설정 (GPT-2 계열 타깃)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8, lora_alpha=16, lora_dropout=0.1,
    target_modules=["attn.c_attn","attn.c_proj","mlp.c_fc","mlp.c_proj"]
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# 5) 긴 한국어 지시문-응답 데이터 (실전 톤, 여러 문장)
long_pairs = [
    (
        "회의 일정 조율 메일의 첫 문단을 정중하고 명확하게 작성해줘. 핵심은 목적, 제안 시간대 2개, 회의 방식(온라인), 다음 단계 안내를 포함하는 것이야.",
        "안녕하세요. 다가오는 제품 기능 점검 관련 회의 일정을 조율드리고자 연락드립니다. 저는 ① 9월 12일(목) 오전 10:00–10:30, ② 9월 12일(목) 오후 2:00–2:30 두 시간대를 우선 제안드립니다. 회의는 온라인(Google Meet)으로 진행할 예정이며, 확정된 일정에 맞춰 접속 링크와 간단한 아젠다를 함께 공유드리겠습니다. 가능하신 시간대를 회신 주시면 감사하겠습니다."
    ),
    (
        "클레임 이메일 초안: 배송 지연에 대한 불만을 정중하지만 단호하게 표현하고, 주문 번호, 지연 기간, 원하는 조치(환불 or 재발송), 응답 기한을 포함해줘.",
        "안녕하세요. 주문번호 #A2024-0911-347 관련하여 배송이 약속된 기한을 5영업일 초과하여 지연되고 있어 문의드립니다. 본 건에 대해 환불 처리 혹은 재발송 중 가능한 조치를 안내해 주시기 바랍니다. 원활한 대응을 위해 9월 15일(일)까지 회신 부탁드립니다. 감사합니다."
    ),
    (
        "비전공자에게 딥러닝을 3–4문장으로 설명해줘. 비유를 하나 포함하고, 과적합을 피하기 위한 기본 아이디어(데이터/정규화)도 간단히 언급.",
        "딥러닝은 많은 예시를 보며 스스로 규칙을 찾아가는 방법으로, 여러 층의 인공 신경망이 데이터를 단계적으로 이해합니다. 사진에서 고양이를 알아보는 일을 여러 번 연습하는 것처럼, 반복 학습을 통해 특징을 스스로 배웁니다. 다만 훈련 데이터에만 지나치게 맞지 않도록 더 많은 데이터와 정규화 같은 기법을 사용해 균형을 맞춥니다. 이는 시험문제만 외우지 않고 개념을 이해하려는 공부법과 비슷합니다."
    ),
    (
        "제품 업데이트 공지 초안: 변경사항 3가지(성능 개선, 다크모드, 오류 수정), 롤아웃 일정, 고객이 해야 할 일(앱 업데이트), 문의 채널을 담아 5문장 내로.",
        "안녕하세요. 이번 업데이트에서는 앱 실행 속도와 반응성이 개선되었고, 많은 요청을 주셨던 다크 모드가 추가되었습니다. 또한 크래시를 유발하던 일부 오류가 수정되어 안정성이 향상되었습니다. 업데이트는 9월 12일부터 순차적으로 적용되며, 앱 스토어에서 최신 버전으로 업데이트하시면 모든 기능을 이용하실 수 있습니다. 문의 사항은 고객센터(help@company.com)로 연락 주시기 바랍니다."
    ),
    (
        "장문의 요약 예시: 다음 단락을 두 문장으로 요약해줘. ‘핵심 수치’와 ‘권고’가 빠지지 않게.",
        "최근 4주간 신규 가입자는 전주 대비 평균 18% 증가했고, 재방문율은 11% 상승했습니다. 이 추세를 유지하려면 퍼널 상단 유입을 안정적으로 확대하는 동시에 온보딩 튜토리얼 개선을 병행할 것을 권고합니다."
    ),
    (
        "개발 일정 보고 메일의 첫 단락: 현재 진행 상황(50%), 남은 범위(알림 모듈/접근성 보완), 위험요인(외부 API 변경), 차주 계획을 4문장 이내로.",
        "안녕하세요. 현재 기능 개발은 약 50% 완료되었으며, 남은 범위는 알림 모듈 구현과 접근성 관련 보완 작업입니다. 외부 결제 API의 사양 변경 가능성이 있는 점이 위험요인으로 판단됩니다. 차주에는 알림 모듈 프로토타입을 완성하고 접근성 점검 체크리스트를 기반으로 개선을 진행하겠습니다."
    ),
    (
        "지원동기 3–4문장: 데이터 기반 문제 해결 경험(로그 분석 → 개선), 협업(디자이너/PM), 합류 후 기여(지표 설정·실험설계)를 강조.",
        "저는 로그 분석을 통해 이탈 구간을 식별하고 온보딩 흐름을 재설계해 전환율을 개선한 경험이 있습니다. 디자이너·PM과 긴밀히 협업하며 목표 지표를 합의하고 빠르게 실험을 반복했습니다. 합류 후에는 핵심 지표를 체계적으로 설정하고 A/B 테스트 설계를 통해 제품의 학습 속도를 높이겠습니다."
    ),
    (
        "리팩터링 제안 메모: 문제(응답 지연/복잡 로직), 원인(중복 쿼리/동기 I/O), 개선(캐싱/비동기화/쿼리 튜닝)과 예상 효과(지연 40%↓)를 5문장 내로.",
        "현재 API 응답 지연과 복잡한 로직으로 유지보수 비용이 증가하고 있습니다. 원인은 중복 데이터 쿼리와 동기 I/O 사용으로 판단됩니다. 캐싱을 도입하고 I/O를 비동기화하며, 쿼리를 튜닝하면 지연 시간을 약 40% 단축할 수 있습니다. 단계별 적용 후 모니터링 대시보드로 효과를 검증하겠습니다."
    ),
    (
        "장문의 안내문: 원격 근무 보안 수칙을 4–5문장으로 작성. VPN 필수, 공용 와이파이 금지, 화면 잠금, 기기 분실 신고 절차 포함.",
        "원격 근무 시 회사 리소스에 접근할 때는 반드시 VPN을 사용해 주세요. 공용 와이파이에서는 접속을 지양하고, 부득이한 경우 개인 핫스팟을 권장합니다. 자리를 비울 때는 화면 잠금을 습관화해 주시고, 기기 분실 시에는 즉시 IT 보안팀에 신고하여 원격 잠금/와이프 절차를 진행합니다. 관련 가이드는 인트라넷 보안 메뉴에서 확인할 수 있습니다."
    ),
]
# 데이터 확장 (반복 + 경미한 변형 없이 샘플 수 늘림)
texts = []
for _ in range(40):  # ≈ 400 샘플 (긴 문장 위주)
    for ins, out in long_pairs:
        texts.append(f"### 지시문:\n{ins}\n\n### 응답:\n{out}{tok.eos_token}")

# 6) Dataset (패딩 라벨 -100 마스킹, 긴 시퀀스 지원)
class TxtDS(Dataset):
    def __init__(self, texts, tok, max_len=512):
        self.texts, self.tok, self.max_len = texts, tok, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = self.tok(
            self.texts[i],
            truncation=True, padding="max_length",
            max_length=self.max_len, return_tensors="pt"
        )
        input_ids = enc["input_ids"].squeeze(0)        # CPU 텐서
        attn      = enc["attention_mask"].squeeze(0)   # CPU 텐서
        labels = input_ids.clone()
        labels[attn == 0] = -100                       # CE ignore_index
        labels = labels.to(torch.long)
        return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

split = int(len(texts) * 0.9)
train_ds = TxtDS(texts[:split], tok, max_len=512)
val_ds   = TxtDS(texts[split:], tok, max_len=512)

# collate: CPU 텐서 유지 (pin_memory와 충돌 방지)
def collate(batch):
    keys = batch[0].keys()
    return {k: torch.stack([b[k] for b in batch]) for k in keys}

# 7) 학습 인자(버전 의존 최소, 안정 우선)
args = TrainingArguments(
    output_dir="./out_kogpt2_lora_long",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,    # 긴 시퀀스 → 배치 작게
    gradient_accumulation_steps=8,    # 유효 배치↑
    learning_rate=2e-4,               # LoRA라 비교적 크게
    logging_steps=25,
    remove_unused_columns=False,
    report_to="none",
    dataloader_pin_memory=False,
    fp16=torch.cuda.is_available(),   # GPU 있으면 fp16, CPU면 자동 False
    bf16=False
)

trainer = Trainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    data_collator=collate, tokenizer=tok
)

print("학습 시작...")
trainer.train()
print("학습 종료.")

# 8) LoRA 어댑터 저장
save_dir = "./lora_adapter_kogpt2_long"
model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)
print("저장:", save_dir)

# 9) 추론 유틸 (긴 응답에 맞춘 샘플링)
@torch.inference_mode()
def generate_ko(prompt, max_new_tokens=220, temperature=0.8, top_p=0.9, top_k=50):
    model.eval()
    prefix = f"### 지시문:\n{prompt}\n\n### 응답:\n"
    inputs = tok(prefix, return_tensors="pt").to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p, top_k=top_k,
        repetition_penalty=1.12,
        no_repeat_ngram_size=3,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id
    )
    txt = tok.decode(out[0], skip_special_tokens=True)
    return txt.split("### 응답:")[-1].strip()

# 10) 데모 (긴 문장 출력 확인)
print("\n=== 데모 ===")
prompts = [
    "회의 일정 조율 메일의 첫 문단을 정중하고 명확하게 작성해줘. 핵심은 목적, 제안 시간대 2개, 회의 방식(온라인), 다음 단계 안내를 포함하는 것이야.",
    "클레임 이메일 초안: 배송 지연에 대한 불만을 정중하지만 단호하게 표현하고, 주문 번호, 지연 기간, 원하는 조치(환불 or 재발송), 응답 기한을 포함해줘.",
    "비전공자에게 딥러닝을 3–4문장으로 설명해줘. 비유를 하나 포함하고, 과적합을 피하기 위한 기본 아이디어(데이터/정규화)도 간단히 언급."
]
for p in prompts:
    print("Q:", p)
    print("A:", generate_ko(p))
    print("-"*40)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 108.2 MB/s eta 0:00:00
Device: cuda | CUDA? True


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

Step,Training Loss
25,3.913400
50,2.735300


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


학습 종료.


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


저장: ./lora_adapter_kogpt2_long

=== 데모 ===
Q: 회의 일정 조율 메일의 첫 문단을 정중하고 명확하게 작성해줘. 핵심은 목적, 제안 시간대 2개, 회의 방식(온라인), 다음 단계 안내를 포함하는 것이야.
A: 장소는 5월 13일(목). 저는 안양대학교 인근 (안녕하세요. 안녕하세요) 공학관 건물로 결정되었습니다. 오늘 오후 3시부터 10분 간격으로 진행될 예정이구요. 연결 플랫폼은 우선 모바일 앱을 이용합니다. 접속 후 회의 일정을 5문장까지 안내할 수 있습니다. 이후 Google Remember Admission과 협업하여 모바일 기기로 참여할 예정입니다” 라고 말했습니다.
이어 “모바일앱에서는 Go game App으로 간단히 접근할 수 있고, 채팅 버튼을 클릭하면 회의 일정을 확인할 수 있습니다.”라고 덧붙였습니다. 박근혜 대통령의 사저 터(서울 종로구 평창동)가 대통령 집무실(원격 근무)를 비롯해 경호ᆞ검색대, 집무실 내 모든 시설을 모두 갖춘 ‘원스톱(One-Stop)’으로 건설된다.
대통령의 국가안보 관련 핵심시설은 안전시설이 가장 큰 비중을 차지한다.
청와대는 22일 국정기획자문위원회로부터 청와대 주요 시설 현황을 보고받고 이같이 밝혔다.
박 대통령은 이 자리에서 지난 17일 열린 수석비서관 회의에서 청와
----------------------------------------
Q: 클레임 이메일 초안: 배송 지연에 대한 불만을 정중하지만 단호하게 표현하고, 주문 번호, 지연 기간, 원하는 조치(환불 or 재발송), 응답 기한을 포함해줘.
A: 해당 메시지는 환불을 포함한 모든 조치를 안내합니다.
저는 지난 5월 26일부터 10월 2일까지 4주간 고객센터와 모바일 앱에서 클레임을 받았습니다. 이 기간 동안 많은 분들이 문의 전화를 받았고, 많은 불편을 끼쳐드려 송구하다”며 사과드립니다.
# 응답 :
본 사안에 대해 9월 3일 안건으로 회신 드립니다. 서울고등법원은 "최근 인터넷 이용자의 약 75%가 불안감을 호소하는 